<a href="https://colab.research.google.com/github/dhanushpachabhatla/My_LangChain_Playground/blob/main/langchain_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lanchain Basics using Gemini API key

In [21]:
import os
from getpass import getpass
os.environ["GOOGLE_API_KEY"] = getpass(
    "Enter OpenAI API Key: "
)

Enter OpenAI API Key: ··········


In [1]:
!pip install langchain

In [2]:
!pip install langchain_google_genai


In [3]:
from langchain_google_genai import GoogleGenerativeAI

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

#Testing different functions to access gemini through langchain

* testing GoogleGenerativeAI

In [10]:
llm = GoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

result = llm.invoke("Summarize LangChain in one line.")
print(result)

LangChain is a framework for developing applications powered by large language models (LLMs).


* testing ChatGoogleGenerativeAI

In [11]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",        # Use "gemini-pro" for text
    temperature=0.3            # Low temperature for concise summaries
)

result = llm.invoke("Summarize LangChain in one line.")
print(result)

content='LangChain is a framework for developing applications powered by large language models (LLMs).' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run--41202fe6-0d76-4ea7-b3f4-cf203f623aae-0' usage_metadata={'input_tokens': 8, 'output_tokens': 18, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}


#Prompt Types

LangChain offers several prompt classes and methods to organize and construct prompts systematically. While we’ll explore these in detail later, let’s focus on the essentials needed now.

Three Core Prompt Types
1. System Prompt

* Provides high-level instructions to the LLM about its role, behavior, and objective.

* Insight: Think of this as “setting the character and boundaries” for the AI.

* Example: "You are a helpful AI assistant. You will do XYZ."

2. User Prompt

* Direct input from the user.

* Insight: Represents dynamic queries or tasks for the LLM to process.

2. AI Prompt

* The AI’s generated response.

* Insight: Previous AI responses and user prompts can be combined as chat history to maintain conversational context.

In [22]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Defining the system prompt (how the AI should act)
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps generate article titles."
)

# the user prompt is provided by the user, in this case however the only dynamic
# input is the article
user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a name for a article.
The article is here for you to examine {article}

The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided.""",
    input_variables=["article"]
)

We can display what our formatted human prompt would look like after inserting a value into the `article` parameter:


In [23]:
user_prompt.format(article="TEST STRING")


HumanMessage(content='You are tasked with creating a name for a article.\nThe article is here for you to examine TEST STRING\n\nThe name should be based of the context of the article.\nBe creative, but make sure the names are clear, catchy,\nand relevant to the theme of the article.\n\nOnly output the article name, no other explanation or\ntext can be provided.', additional_kwargs={}, response_metadata={})

We have our system and user prompts, we can merge both into our full chat prompt using the ChatPromptTemplate

In [24]:
from langchain.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

By default, the `ChatPromptTemplate` will read the `input_variables` from each of the prompt templates inserted and allow us to use those input variables when formatting the full chat prompt template:

`ChatPromptTemplate` also prefixes each individual message with it's role, ie `System:`, `Human:`, or `AI:`.

In [25]:
print(first_prompt.format(article="TEST STRING"))

System: You are an AI assistant that helps generate article titles.
Human: You are tasked with creating a name for a article.
The article is here for you to examine TEST STRING

The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided.


In [16]:
text = """About the work from home job/internship
This isn't just "another internship." At our company, you'll work with real business data that drives real strategies not dummy projects or case studies. If you've been waiting for the right launchpad to kickstart your career in analytics, this is the opportunity you've been looking for.

Selected intern's day-to-day responsibilities include:

1. Collect, clean, and organize raw data from multiple business sources.
2. Analyze large datasets using Excel, Google Sheets, or SQL to uncover meaningful insights.
3. Create impactful dashboards and visualizations in Power BI, Tableau, or Google Data Studio.
4. Collaborate with teams across marketing, operations, and product to support data-driven decisions.
5. Identify trends, anomalies, and patterns that influence key business strategies.
6. Ensure accuracy, consistency, and reliability in all reports and dashboards.
7. Present insights clearly and concisely to both technical and non-technical stakeholders.
8. Stay updated with emerging tools, techniques, and best practices in analytics to strengthen your skill set.
Skill(s) required
Data Analysis
Data Analytics
Data Interpretation
ETL processes
MS-Excel
MySQL
Pandas
Pivot Table
Power BI
Python
SQL
Statistics
Tableau
Earn certifications in these skills
Learn Google Analytics
Learn MS-Excel
Learn Python
Learn Statistical Modeling
Learn Data Analytics
Learn MySQL
Learn SQL
Learn PostgreSQL
Learn MS SQL Server
Learn Power BI
Who can apply

Only those candidates can apply who:

1. are available for the work from home job/internship

2. can start the work from home job/internship between 28th Aug'25 and 2nd Oct'25

3. are available for duration of 3 months

4. have relevant skills and interests

Perks
Certificate
Letter of recommendation
Flexible work hours
5 days a week
Job offer
Additional information
Job offer:
On successful conversion to a permanent employee, the candidate can expect a salary of ₹ 600000 to 800000/year
Number of openings
5
About APEX ENGINEERING
Apex Engineering is a service provider, delivering high-quality engineering and IT services across the globe. With expertise in domains like core engineering, CAD-based design, utility surveys, data processing, and emerging IT solutions, we serve clients ranging from top infrastructure firms to growing tech-driven enterprises."""

#Chaining concept

LangChain Expression Language `LCEL` allows you to compose sequences of `runnables` (like prompts, language models, or output parsers) to form a chain. This is a powerful way to build complex applications by combining simple, reusable components. LCEL makes chaining easier, more robust, and more efficient. ⛓️

Key Features of LCEL :
1. Asynchronous Support: You can run parts of a chain asynchronously, which is crucial for building scalable applications that don't block.

2. Streaming: LCEL supports streaming, allowing you to get a response piece by piece, which is great for building real-time applications like chatbots.

3. Parallelism: Multiple parts of a chain can run in parallel, significantly speeding up execution.

4. Fallback: You can define fallback mechanisms, so if one runnable fails, the chain can automatically try another.

5. Input and Output Schema: LCEL components have a defined input and output schema, making it easy to know what to expect and ensuring compatibility between components.

In [19]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import PromptTemplate

# Step 1: Prompt Template
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following text in 100 words:\n{text}"
)

# Step 2: Custom Post-Processing Function
def format_summary(summary):
    return f"Final Summary: {summary}"

format_runnable = RunnableLambda(format_summary)

# Step 3: LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3
)


# Step 4: Chain
summary_chain = (
    {"text": lambda x: x["text"]}
    | summary_prompt
    | llm
    | (lambda x: x.content)
    | format_runnable
)


In [20]:
summary_generated = summary_chain.invoke({"text" : text})
summary_generated

'Final Summary: This remote analytics internship at Apex Engineering offers hands-on experience using real business data.  Interns will collect, clean, analyze data (Excel, SQL, Python), create dashboards (Power BI, Tableau), collaborate with various teams, and present findings.  The 3-month internship (Aug 28th - Oct 2nd), requires data analysis skills and offers a certificate, letter of recommendation, flexible hours, and potential for a ₹600,000-800,000/year job. Five positions are available.'